# Árvore de decisão


Ilustra o funcionamento do algoritmo de árvore de decisão com dados contínuos.

Este notebook foi desenvolvido para o ambiente GOOGLE COLAB ([colab.research.google.com](https://colab.research.google.com)).

Prof. Hugo de Paula

-------------------------------------------------------------------------------

### Base de dados: Sonar, Mines vs. Rocks

https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar,+Mines+vs.+Rocks%29

208 instâncias

60 atributos

2 classes (rocha, mina)



In [ ]:
!pip install pydotplus
!pip install dtreeviz

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, tree

# funcao que permite partir base de dados em treinamento e teste
from sklearn.model_selection import train_test_split

# modelo que vai ser treinado
from sklearn.tree import DecisionTreeClassifier

# calcula acuracia do modelo e relatorio de recall, precision e FScore
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer

# permite transformar string em classe binaria
from sklearn.preprocessing import LabelEncoder

from google.colab import files
uploaded = files.upload()


### Carga dos dados e particionamento das bases de treinamento e teste

### Transformação de dados

A classe é convertida para labels únicos sequenciais.

<code>
 le = preprocessing.LabelEncoder()
  
 le.fit(dados)
</code>


### Particionamento da base

<code>train_test_split(X, y) -- particiona a base de dados original em bases de treinamento e teste.</code>

No código a seguir, são utilizados 10% para teste e 90% para treinamento.







In [ ]:
# leitura da base de dados e estatistica 
# console descreve dimensao da base
# descricao estatistica de cada atributo
# nao existe dado omisso (count), o que e bom
sonar = pd.read_excel('sonar.xlsx', sheet_name=0) 
print("\nDimensões: {0}".format(sonar.shape))
print("\nCampos: {0}".format(sonar.keys()))
print(sonar.describe(), sep='\n')

# binalizar a classe
# iloc permite slicing
X = sonar.iloc[:,0:(sonar.shape[1] - 1)]

# pegar classe nao binaria
# transformar coluna, codificar coluna
le = LabelEncoder()
y = le.fit_transform(sonar.iloc[:,(sonar.shape[1] - 1)])

# Particiona a base de dados e gerar 4 atributos
# random_state = 0 para sempre iniciar da mesma forma
# 75% treinamento, 25% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)




### Indução do Modelo


Os três passos para indução de um modelo são:

1.   Instanciar o modelo: ``` DecisionTreeClassifier()```
2.   Treinar o modelo: ```fit()```
3.   Testar o modelo: ```predict()```



In [ ]:
# induzir classe de decisao
# entropia: ganho da informacao
sonar_tree = DecisionTreeClassifier(random_state=0, criterion='entropy')
# comando fit treina
sonar_tree = sonar_tree.fit(X_train, y_train)
# calculo acuracia
print("Acurácia (base de treinamento):", sonar_tree.score(X_train, y_train))

# tem que dar mesmo resultado de score do de cima
# utiliza teste para predicao
y_pred = sonar_tree.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["Mina", "Rocha"]))

# quando se usa teste e treinamento, acuracia tende a cair
#

cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix, index=["Mina", "Rocha"], columns=["Mina (prev)", "Rocha (prev)"])
print(cnf_table)



### Exibição da árvore de decisão

In [ ]:
from dtreeviz.trees import *

viz = dtreeviz(sonar_tree,
              X_train,
              y_train,
              target_name="Classe",
              feature_names=np.arange(0,60),
              class_names=["mina", "rocha"])  

viz.view()

In [ ]:
import pydotplus 
# Create DOT data
dot_data = tree.export_graphviz(sonar_tree, out_file=None, 
                                proportion=False,
                                rounded =True,
                                filled=True,
                                feature_names=np.arange(0,60),  
                                class_names=["mina", "rocha"])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)  

# Show graph
Image(graph.create_png())
